In [1]:
import numpy as np
import pandas as pd

def h(x,w):
  return np.sign(np.dot(x,w))

def holdout(data, trainp, validp):
  m = np.shape(data)[0]

  train = data[0 : int(np.floor(m*trainp/100))]
  valid = data[int(np.floor(m*trainp/100)) : int(np.floor(m*trainp/100))+int(np.floor(m*validp/100))]
  test = data[int(np.floor(m*trainp/100))+int(np.floor(m*validp/100)) : None]

  y_train = train[:,-1] #shape = (rows,)
  x_train = np.delete(train, -1, axis=1)

  y_valid = valid[:,-1] #shape = (rows,)
  x_valid = np.delete(valid, -1, axis=1)

  y_test = test[:,-1] #shape = (rows,)
  x_test = np.delete(test, -1, axis=1)

  return x_train,x_valid,x_test,y_train,y_valid,y_test

def appendones(x):
  m = np.shape(x)[0]
  return np.concatenate((np.ones((m,1)),x), axis=1)

def conf_matrix(y,pred):
  m = np.zeros(4)
  for i in range(len(y)):
    if y[i]==-1 and pred[i]==-1:
      m[0] += 1
    if y[i]==-1 and pred[i]==1:
      m[1] += 1
    if y[i]==1 and pred[i]==-1:
      m[2] += 1
    if y[i]==1 and pred[i]==1:
      m[3] += 1
  return m

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/NNFL Assignments (Aug 2021)/Assignment 2/data55.xlsx')
cols = np.array(data.columns, ndmin=2)
data = data.to_numpy()
data = np.concatenate((cols,data), axis=0)

# shuffle & train-test-valid-split
np.random.shuffle(data)
x_train, x_valid, x_test, y_train, y_valid, y_test = holdout(data, 70, 10)

# # normalizing
# mu = np.mean(x_train, axis=0)
# std = np.std(x_train, axis=0)

# x_train = (x_train-mu)/std
# x_valid = (x_valid-mu)/std
# x_test = (x_test-mu)/std

# appending ones (input for bias)
x_train = appendones(x_train)
x_valid = appendones(x_valid)
x_test = appendones(x_test)

n = np.shape(x_train)[1]
m_train = np.size(y_train)
m_valid = np.size(y_valid)
m_test = np.size(y_test)

# changing classes from {0,1} to {-1,1}
y_train = np.where(y_train==0, -1, 1)
y_valid = np.where(y_valid==0, -1, 1)
y_test = np.where(y_test==0, -1, 1)

In [ ]:
# grid search
alpha_grid = list(np.linspace(0.01, 0.1, 50))
T_grid = [300, 350, 400, 450, 500]

accuracy_vals = np.zeros((len(T_grid),len(alpha_grid)))
for T in T_grid:
  for alpha in alpha_grid:
    w = np.zeros(n)
    for t in range(T):
      for i in range(m_train):
        y_pred = h(x_train[i,:],w)
        if y_pred != y_train[i]:
          w = w + alpha*y_train[i]*x_train[i,:]
    pred = h(x_valid,w)
    tn,fp,fn,tp = conf_matrix(y_valid, pred)
    accuracy_vals[T_grid.index(T)][alpha_grid.index(alpha)] = (tp+tn)/(tp+tn+fp+fn)

# index of maximum accuracy
index = np.unravel_index(np.argmax(accuracy_vals, axis=None), accuracy_vals.shape)

print("maximum validation accuracy = {}".format(accuracy_vals[index]))
print("index = {}".format(index))
print("Optimal T value = {}\nOptimal alpha value = {}".format(T_grid[index[0]],alpha_grid[index[1]]))

maximum validation accuracy = 0.75
index = (2, 0)
Optimal T value = 400
Optimal alpha value = 0.01


In [ ]:
w = np.zeros(n) # zero initial weights
alpha = 0.01
T = 400

# training
for t in range(T):
  for i in range(m_train):
    y_pred = h(x_train[i,:],w)
    if y_pred != y_train[i]:
      w = w + alpha*y_train[i]*x_train[i,:]

# testing
pred = h(x_test,w)
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.7209302325581395
Sensitivity = 0.8333333333333334
Specificity = 0.5789473684210527
